<a href="https://colab.research.google.com/github/Jaehooni/2022_Summer_D.Com_DeepLearning_Study/blob/main/lab06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Imports**

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

**Softmax**
$$ P(class=i) = \frac{e^i}{\sum e^i} $$

In [2]:
z = torch.FloatTensor([1, 2, 3])

In [3]:
hypothesis = F.softmax(z, dim=0)
print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


In [4]:
hypothesis.sum()

tensor(1.)

**Cross Entropy Loss**
$$ L = \frac{1}{N} \sum - y \log(\hat{y}) $$
<br>
두개의 확률 분포가 주어졌을 때 두 확률 분포의 유사성을 체크
<br>
활용: Cross Entropy를 minimize하는 방식으로 모델 최적화

**Cross Entropy Loss (Low-level)**

In [5]:
z = torch.rand(3, 5, requires_grad=True)
hypothesis = F.softmax(z, dim=1)
print(hypothesis)

tensor([[0.2310, 0.1295, 0.2556, 0.2505, 0.1334],
        [0.1316, 0.2025, 0.1826, 0.2163, 0.2670],
        [0.1377, 0.2026, 0.1353, 0.3415, 0.1829]], grad_fn=<SoftmaxBackward0>)


In [6]:
y = torch.randint(5, (3,)).long()
print(y)

tensor([1, 0, 3])


In [7]:
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

tensor([[0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0.]])

In [8]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.7155, grad_fn=<MeanBackward0>)


**Cross-entropy Loss with torch.nn.functional**

<small> PyTorch has `F.log_softmax()` function </small>

In [9]:
# Low level
torch.log(F.softmax(z, dim=1))

tensor([[-1.4655, -2.0440, -1.3642, -1.3842, -2.0143],
        [-2.0280, -1.5969, -1.7003, -1.5311, -1.3206],
        [-1.9826, -1.5966, -2.0002, -1.0744, -1.6989]], grad_fn=<LogBackward0>)

In [10]:
# High level
F.log_softmax(z, dim=1)

tensor([[-1.4655, -2.0440, -1.3642, -1.3842, -2.0143],
        [-2.0280, -1.5969, -1.7003, -1.5311, -1.3206],
        [-1.9826, -1.5966, -2.0002, -1.0744, -1.6989]],
       grad_fn=<LogSoftmaxBackward0>)

<small> PyTorch also has `F.nll_loss()` function that computes the negative loss likelihood. </small>

In [11]:
# Low level
(y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean()

tensor(1.7155, grad_fn=<MeanBackward0>)

In [12]:
# High level
F.nll_loss(F.log_softmax(z, dim=1), y)
#NLL = Negative Log Likelihood

tensor(1.7155, grad_fn=<NllLossBackward0>)

<small> PyTorch also has `F.cross_entropy` that combines `F.log_softmax()` and `F.nll_loss()`. </small>

In [13]:
F.cross_entropy(z, y)

tensor(1.7155, grad_fn=<NllLossBackward0>)

**Training with Low-level Cross Entropy Loss**

In [14]:
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train) 

In [15]:
# 모델 초기화
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost 계산 (1)
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1) # or .mm or @
    y_one_hot = torch.zeros_like(hypothesis)
    y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
    cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean()

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.901535
Epoch  200/1000 Cost: 0.839114
Epoch  300/1000 Cost: 0.807826
Epoch  400/1000 Cost: 0.788472
Epoch  500/1000 Cost: 0.774822
Epoch  600/1000 Cost: 0.764449
Epoch  700/1000 Cost: 0.756191
Epoch  800/1000 Cost: 0.749398
Epoch  900/1000 Cost: 0.743671
Epoch 1000/1000 Cost: 0.738749


**Training with F.cross_entropy**

In [17]:
# 모델 초기화
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost 계산 (2)
    z = x_train.matmul(W) + b # or .mm or @
    cost = F.cross_entropy(z, y_train) # scatter 사용 X

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568256
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962


**High-level Implementation with nn.Module**

In [18]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 3) # Output이 3!

    def forward(self, x):
        return self.linear(x)

In [19]:
model = SoftmaxClassifierModel()

In [20]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.cross_entropy(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 20번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 5.977555
Epoch  100/1000 Cost: 0.631481
Epoch  200/1000 Cost: 0.549811
Epoch  300/1000 Cost: 0.496067
Epoch  400/1000 Cost: 0.452249
Epoch  500/1000 Cost: 0.413131
Epoch  600/1000 Cost: 0.376311
Epoch  700/1000 Cost: 0.340259
Epoch  800/1000 Cost: 0.303876
Epoch  900/1000 Cost: 0.267380
Epoch 1000/1000 Cost: 0.241519


딥러닝을 할때 일반적으로 Classification을 하기 위해서는 `Cross Entropy`와 `Softmax`를 사용함
<br>
<br>
하지만 Binary Classfication일 경우에는 logistic regression에서 사용한 loss 함수
<br>
=> `BCE(Binary Cross Entrophy)`와 `sigmoid`를 사용
